In [172]:
import nltk
from nltk.corpus import *
import string
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import wikipedia as wp
from nltk.util import ngrams

__◑ Read in some text from a corpus, tokenize it, and print the list of all wh-word types that occur. (wh-words in English are used in questions, relative clauses and exclamations: who, which, what, and so on.) Print them in order. Are any words duplicated in this list, because of the presence of case distinctions or punctuation?__

Specifying only startwith WH gets unrelated words such as "whisper". In my case they are not duplicated due to case or punctuation becuase the text was pre-processed before extracting WH. 

In [5]:
emma = gutenberg.raw("austen-emma.txt")
emma_tokens = nltk.word_tokenize(emma)

In [7]:
wh = [tok.lower() for tok in emma_tokens if tok.lower().startswith("wh")]
set(wh)

{'what',
 'whatever',
 'wheat',
 'when',
 'whence',
 'whenever',
 'where',
 'wheres',
 'wherever',
 'whether',
 'which',
 'whichever',
 'while',
 'whim',
 'whims',
 'whirling',
 'whisper',
 'whispered',
 'whispering',
 'whispers',
 'whist',
 'whist-club',
 'whist-players',
 'white',
 'whiten',
 'who',
 'whoever',
 'whole',
 'whole-length',
 'whole-lengths',
 'wholesome',
 'wholesome.',
 'wholesomeness',
 'wholly',
 'whom',
 'whose',
 'why'}

__◑ Create a file consisting of words and (made up) frequencies, where each line consists of a word, the space character, and a positive integer, e.g. fuzzy 53. Read the file into a Python list using open(filename).readlines(). Next, break each line into its two fields using split(), and convert the number into an integer using int(). The result should be a list of the form: [['fuzzy', 53], ...].__

In [14]:
lines = []
with open("freqs.txt") as f:
    for line in f.readlines():
        lines.append(line.split())

In [15]:
lines

[['fuzzy', '54'],
 ['wug', '101'],
 ['nlp', '12'],
 ['machine', '4'],
 ['learning', '8'],
 ['corpus', '90']]

__◑ Write code to access a favorite webpage and extract some text from it. For example, access a weather site and extract the forecast top temperature for your town or city today.__

In [38]:
def get_url(url):
    import requests
    from bs4 import BeautifulSoup
    from unicodedata import normalize
    raw = requests.get(url).text
    clean1 = BeautifulSoup(raw)
    final = " ".join([s for s in clean1.stripped_strings])
    return normalize("NFKC", final)

In [42]:
odyssey = get_url("https://www.gutenberg.org/files/3160/3160-h/3160-h.htm")
odyssey[:100]

'The Project Gutenberg eBook of The Odyssey, by Homer The Project Gutenberg eBook of The Odyssey, by '

__◑ Write a function unknown() that takes a URL as its argument, and returns a list of unknown words that occur on that webpage. In order to do this, extract all substrings consisting of lowercase letters (using re.findall()) and remove any items from this set that occur in the Words Corpus (nltk.corpus.words). Try to categorize these words manually and discuss your findings.__

In [72]:
def preprocess(text):
    if type(text) == str:
        text = nltk.word_tokenize(text)
        text = [w.lower() for w in text if w.lower() not in stopwords.words("english") and w.lower() not in string.punctuation]
        
    return text

In [52]:
def unknown(url):
    text = get_url(url)
    corpus = [w.lower() for w in words.words()]
    unknown = [w for w in preprocess(text) if w not in corpus]
    return unknown

__◑ Examine the results of processing the URL http://news.bbc.co.uk/ using the regular expressions suggested above. You will see that there is still a fair amount of non-textual data there, particularly Javascript commands. You may also find that sentence breaks have not been properly preserved. Define further regular expressions that improve the extraction of text from this web page.__

In [55]:
get_url("http://news.bbc.co.uk/")[:200] # firts 200 char, seems to be quite clean

'Home - BBC News Homepage Accessibility links Skip to content Accessibility Help BBC Account Notifications Home News Sport Weather iPlayer Sounds CBBC CBeebies Food Bitesize Arts Taster Local Three Men'

In [56]:
get_url("http://news.bbc.co.uk/")[-200:] # last 200 char, also seems to be quite clean

'ity Help Parental Guidance Contact the BBC Get Personalised Newsletters Copyright © 2022 BBC. The BBC is not responsible for the content of external sites. Read about our approach to external linking.'

__◑ Are you able to write a regular expression to tokenize text in such a way that the word _don't_ is tokenized into _do_ and _n't_? Explain why this regular expression won't work: «n't|\w+».__

It does word due to the greedy quantifier \w+. Making the quantifier lazy (\w+?) will not solve the problem, since it will match as less as possible, matching individual characters

In [2]:
re.findall(r'n\'t|\w+(?=n\'t)|\w+(?!n\'t)', "I don't want to go! I won't go. 'Quotes' (citation, 2021)")

['I',
 'do',
 "n't",
 'want',
 'to',
 'go',
 'I',
 'wo',
 "n't",
 'go',
 'Quotes',
 'citation',
 '2021']

__◑ Try to write code to convert text into hAck3r, using regular expressions and substitution, where e → 3, i → 1, o → 0, l → |, s → 5, . → 5w33t!, ate → 8. Normalize the text to lowercase before converting it. Add more substitutions of your own. Now try to map s to two different values: $ for word-initial s, and 5 for word-internal s.__

In [59]:
def translate(word):
    org = ['ate', 'e', 'i', 'o', 'l', '\.']
    sub = ['8', '3', '1', '0', '|', '5w33t!']
    for i in range(len(org)):
        word = re.sub(org[i], sub[i], word.lower())
        word = re.sub("^s", "$", word.lower())
        word = re.sub("(?<!^)s", "5", word.lower())
    print(word)

In [61]:
translate("String with s internal - salmon")

$tr1ng w1th 5 1nt3rna| - 5a|m0n


__◑ Pig Latin is a simple transformation of English text. Each word of the text is converted as follows: move any consonant (or consonant cluster) that appears at the start of the word to the end, then append ay, e.g. string → ingstray, idle → idleay. http://en.wikipedia.org/wiki/Pig_Latin__

Write a function to convert a word to Pig Latin.

Write code that converts text, instead of individual words.

Extend it further to preserve capitalization, to keep qu together (i.e. so that quiet becomes ietquay), and to detect when y is used as a consonant (e.g. yellow) vs a vowel (e.g. style).

In [88]:
def pig_latin(tokens):
    
    if type(tokens) == str:
        tokens = nltk.word_tokenize(tokens.lower())
    new_word = []

    for word in tokens:
        if re.search("^[^a-z]", word.lower()):
            new_word.append(word)
        elif re.search("^[aoiue]", word.lower()):
            new_word.append(word + "ay")
        elif re.search("^[^aoiue]", word.lower()):
            d = re.findall("(^[^aouie]+)([aoiue])", word)
            start = word.index(d[0][1])
            new_word.append(word[start:] + d[0][0] + "ay")
        
    return new_word

In [92]:
pig_latin("tokens and some more words plus 700")

['okenstay', 'anday', 'omesay', 'oremay', 'ordsway', 'usplay', '700']

__◑ Download some text from a language that has vowel harmony (e.g. Hungarian), extract the vowel sequences of words, and create a vowel bigram table.__


In [93]:
turk_text = get_url("https://www.bbc.com/turkce/haberler-turkiye-60593781")

In [101]:
vowels_in_word = []
for word in turk_text.split():
    vowels_in_word.append(re.findall("[aeıioöuü]", word))

In [123]:
bi_dict = defaultdict(int)
for vow_seq in vowels_in_word:
    for bi in nltk.bigrams(vow_seq):
        bi_dict[bi] += 1

In [125]:
len(bi_dict) # all possible vow bigrams in the tex

36

In [131]:
a = np.zeros((3,3))

__◑ Python's random module includes a function choice() which randomly chooses an item from a sequence, e.g. choice("aehh ") will produce one of four possible characters, with the letter h being twice as frequent as the others. Write a generator expression that produces a sequence of 500 randomly chosen letters drawn from the string "aehh ", and put this expression inside a call to the ''.join() function, to concatenate them into one long string. You should get a result that looks like uncontrolled sneezing or maniacal laughter: he  haha ee  heheeh eha. Use split() and join() again to normalize the whitespace in this string.__

In [134]:
"".join([random.choice("aehh") for _ in range(50)]) #sneezing

'aehhhhhheaehheehhhaeehhhhehehehhahhehaaahehheehhae'

In [135]:
"".join([random.choice("aehh ") for _ in range(50)]) #some whitespace

'ah h hh haeeh eehhh h hh  h h hhheheehaheehh hehea'

__◑ Consider the numeric expressions in the following sentence from the MedLine Corpus: The corresponding free cortisol fractions in these sera were 4.53 +/- 0.15% and 8.16 +/- 0.23%, respectively. Should we say that the numeric expression 4.53 +/- 0.15% is three words? Or should we say that it's a single compound word? Or should we say that it is actually nine words, since it's read "four point five three, plus or minus zero point fifteen percent"? Or should we say that it's not a "real" word at all, since it wouldn't appear in any dictionary? Discuss these different possibilities. Can you think of application domains that motivate at least two of these answers?__



__◑ Readability measures are used to score the reading difficulty of a text, for the purposes of selecting texts of appropriate difficulty for language learners. Let us define μw to be the average number of letters per word, and μs to be the average number of words per sentence, in a given text. The Automated Readability Index (ARI) of the text is defined to be: 4.71 μw + 0.5 μs - 21.43. Compute the ARI score for various sections of the Brown Corpus, including section f (lore) and j (learned). Make use of the fact that nltk.corpus.brown.words() produces a sequence of words, while nltk.corpus.brown.sents() produces a sequence of sentences.__



In [107]:
def ari(words, sents):
    s = sum([len(s) for s in sents]) / len(sents)
    w = sum([len(w) for w in words]) / len(words)
    ari = (4.71 * w) + (0.5 * s) - 21.43
    return ari

for cat in brown.categories():
    ari_score = ari(brown.words(categories=cat), brown.sents(categories=cat))
    print("{:.2f} {:<20s}".format(ari_score, cat))

4.08 adventure           
10.99 belles_lettres      
9.47 editorial           
4.91 fiction             
12.08 government          
8.92 hobbies             
7.89 humor               
11.93 learned             
10.25 lore                
3.83 mystery             
10.18 news                
10.20 religion            
10.77 reviews             
4.35 romance             
4.98 science_fiction     


__◑ Use the Porter Stemmer to normalize some tokenized text, calling the stemmer on each word. Do the same thing with the Lancaster Stemmer and see if you observe any differences.__

In [162]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()

In [183]:
news = brown.words(categories="news")
news_porter = [porter.stem(w) for w in news]
news_lanc = [lancaster.stem(w) for w in news]

In [184]:
print(*news_porter[:20])
print(*news_lanc[:20])
print(*news[:20])

the fulton counti grand juri said friday an investig of atlanta' recent primari elect produc `` no evid '' that
the fulton county grand jury said friday an investig of atlanta's rec prim elect produc `` no evid '' that
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that


__◑ Define the variable saying to contain the list ['After', 'all', 'is', 'said', 'and', 'done', ',', 'more',
'is', 'said', 'than', 'done', '.'].__

__Process this list using a for loop, and store the length of each word in a new list lengths. Hint: begin by assigning the empty list to lengths, using lengths = []. Then each time through the loop, use append() to add another length value to the list. Now do the same thing using a list comprehension.__

In [185]:
list1 = ['After', 'all', 'is', 'said', 'and', 'done', ',', 'more', 'is', 'said', 'than', 'done', '.']
length_list1 = [len(w) for w in list1]
length_list1

[5, 3, 2, 4, 3, 4, 1, 4, 2, 4, 4, 4, 1]

__◑ Define a variable silly to contain the string: 'newly formed bland ideas are inexpressible in an infuriating
way'. (This happens to be the legitimate interpretation that bilingual English-Spanish speakers can assign to Chomsky's famous nonsense phrase, colorless green ideas sleep furiously according to Wikipedia). Now write code to perform the following tasks:__

Split silly into a list of strings, one per word, using Python's split() operation, and save this to a variable called bland.

Extract the second letter of each word in silly and join them into a string, to get 'eoldrnnnna'.

Combine the words in bland back into a single string, using join(). Make sure the words in the resulting string are separated with whitespace.

Print the words of silly in alphabetical order, one per line.

In [187]:
silly = "newly formed bland ideas are inexpressible in an infuriating way"
split_silly = silly.split()  # split
second_letters = "".join([w[1] for w in split_silly])  # extract 2nd letter
joined_silly = " ".join(split_silly)  # join again
sorted(split_silly)  #sort alpahbetically

['an',
 'are',
 'bland',
 'formed',
 'ideas',
 'in',
 'inexpressible',
 'infuriating',
 'newly',
 'way']

__◑ The index() function can be used to look up items in sequences. For example, 'inexpressible'.index('e') tells us the index of the first position of the letter e.__

What happens when you look up a substring, e.g. 'inexpressible'.index('re')?

Define a variable words containing a list of words. Now use words.index() to look up the position of an individual word.

Define a variable silly as in the exercise above. Use the index() function in combination with list slicing to build a list phrase consisting of all the words up to (but not including) in in silly.

In [192]:
"inexpressible"["inexpressible".index('re')]  # gives the beginning index of "re"

'r'

In [195]:
list1[list1.index("said")]

'said'

In [196]:
silly[:silly.index("in")]

'newly formed bland ideas are '

__◑ Write code to convert nationality adjectives like Canadian and Australian to their corresponding nouns Canada and Australia (see http://en.wikipedia.org/wiki/List_of_adjectival_forms_of_place_names).__

In [ ]:
def nationality(adj):
    """
    Converts nationality adjectives like Canadian and Australian to their corresponding nouns Canada and Australia
    """
    html = wp.page("List_of_adjectival_and_demonymic_forms_of_place_names").html().encode("UTF-8")
    for i in range(1, 13):
        try: 
            df = pd.read_html(html)[i]
        except IndexError:
            break
            
    df.Adjective
    
    

__◑ Read the LanguageLog post on phrases of the form as best as p can and as best p can, where p is a pronoun. Investigate this phenomenon with the help of a corpus and the findall() method for searching tokenized text described in 3.5. http://itre.cis.upenn.edu/~myl/languagelog/archives/002733.html__

In [269]:
tokens_brown = brown.words()
fd = nltk.FreqDist(match for match in re.findall(r'as best (?:as |)(?:he|she|you|I|we|they) can', " ".join(tokens_brown)))

In [270]:
# the above fd did not give any results, so I will try wihtout the first "as"
re.findall(r'best (?:as )?(?:he|she|you|I|we|they) can', " ".join(tokens_brown))

['best they can']

In [271]:
# another corpus, still no matches with "as"
re.findall(r'(?:as )?best (?:as )?(?:he|she|you|I|we|they) can', " ".join(gutenberg.words()))

['best we can', 'best I can']

__◑ Study the lolcat version of the book of Genesis, accessible as nltk.corpus.genesis.words('lolcat.txt'), and the rules for converting text into lolspeak at http://www.lolcatbible.com/index.php?title=How_to_speak_lolcat. Define regular expressions to convert English words into corresponding lolspeak words.__

I will skip this one, as the url does not open and I couldn't find lolcat rules

In [272]:
nltk.corpus.genesis.words('lolcat.txt')

['Oh', 'hai', '.', 'In', 'teh', 'beginnin', 'Ceiling', ...]

__◑ Read about the re.sub() function for string substitution using regular expressions, using help(re.sub) and by consulting the further readings for this chapter. Use re.sub in writing code to remove HTML tags from an HTML file, and to normalize whitespace.__

In [277]:
def html_clean(html):
    html = re.sub(r'<\/?\w+?>', "", html) #remove html markups
    html = re.sub(r'(\s|\n)+', " ", html) #remove extra whitespaces
    return html

__★ An interesting challenge for tokenization is words that have been split across a line-break. E.g. if long-term is split, then we have the string long-\nterm.__

Write a regular expression that identifies words that are hyphenated at a line-break. The expression will need to include the \n character.

Use re.sub() to remove the \n character from these words.

How might you identify words that should not remain hyphenated once the newline is removed, e.g. 'encyclo-\npedia'?x


In [33]:
line_break = ""
with open("linbreaks.txt", "r") as f:
    for line in f.readlines():
        line_break += line
print(line_break)

encyclo-
pedia
or it can be a self-
esteem
maybe long-
term or something incompre-
hensible


In [34]:
line_break = re.sub("(?<=-)\n", "", line_break)
line_break = re.sub("\n(?=[a-zA-Z])", " ", line_break)
line_break

'encyclo-pedia or it can be a self-esteem maybe long-term or something incompre-hensible'

In [38]:
after_join = []
for w in line_break.split():
    if w.replace("-","") in nltk.corpus.words.words():
        after_join.append(w.replace("-",""))
    else:
        after_join.append(w)

print(" ".join(after_join))

encyclopedia or it can be a self-esteem maybe long-term or something incomprehensible


__★ Read the Wikipedia entry on Soundex. Implement this algorithm in Python.__

In [58]:
mappings = {let:num for num, let in zip("01230120022455012623010202", "abcdefghijklmnopqrstuvwxyz")}

In [103]:
def soundex(word):
    
    soundex = ""
    soundex += word[0].upper()
    word = word.lower()
    
    for i in range(1, len(word)):
        if mappings[word[i]] != "0": soundex += mappings[word[i]] 
    
    if len(soundex) < 4:
        soundex += (4 - len(soundex)) * "0"
    elif len(soundex) > 3:
        soundex = soundex[:4]
    return soundex

In [104]:
print(soundex("SIM"), soundex("Davis"), soundex("Richard"), soundex("Rupert"))

S500 D120 R263 R163


__★ Obtain raw texts from two or more genres and compute their respective reading difficulty scores as in the earlier exercise on reading difficulty. E.g. compare ABC Rural News and ABC Science News (nltk.corpus.abc). Use Punkt to perform sentence segmentation.__

In [53]:
for file in abc.fileids():
    print("{:.2f} {:>15s}".format(ari(abc.words(file), abc.sents(file)), file))

12.35       rural.txt
12.53     science.txt


__★ Use WordNet to create a semantic index for a text collection. Extend the concordance search program in 3.6, indexing each word using the offset of its first synset, e.g. wn.synsets('dog')[0].offset (and optionally the offset of some of its ancestors in the hypernym hierarchy).__

In [123]:
from nltk.corpus import wordnet as wn
class IndexedText():
    def __init__(self, stemmer, text):
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
            for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = int(width/4) # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            offset = '(WordNet Offset: ' + str(wn.synsets(self._text[i])[0].offset()) + ')'
            ldisplay = '%*s' % (width, lcontext[-width:])
            rdisplay = '%-*s' % (width, rcontext[:width])
            print(ldisplay, rdisplay, offset)
                
    def _stem(self, word):
        return self._stemmer.stem(word).lower()

porter = nltk.PorterStemmer()
grail = nltk.corpus.webtext.words('grail.txt')
text = IndexedText(porter, grail)
text.concordance('lie')

r king ! DENNIS : Listen , strange women lying in ponds distributing swords is no (WordNet Offset: 751944)
 beat a very brave retreat . ROBIN : All lies ! MINSTREL : [ singing ] Bravest of (WordNet Offset: 6756831)
       Nay . Nay . Come . Come . You may lie here . Oh , but you are wounded !    (WordNet Offset: 6756831)
doctors immediately ! No , no , please ! Lie down . [ clap clap ] PIGLET : Well   (WordNet Offset: 6756831)
ere is much danger , for beyond the cave lies the Gorge of Eternal Peril , which  (WordNet Offset: 6756831)
   you . Oh ... TIM : To the north there lies a cave -- the cave of Caerbannog -- (WordNet Offset: 6756831)
h it and lived ! Bones of full fifty men lie strewn about its lair . So , brave k (WordNet Offset: 6756831)
not stop our fight ' til each one of you lies dead , and the Holy Grail returns t (WordNet Offset: 6756831)


__★ With the help of a multilingual corpus such as the Universal Declaration of Human Rights Corpus (nltk.corpus.udhr), and NLTK's frequency distribution and rank correlation functionality (nltk.FreqDist, nltk.spearman_correlation), develop a system that guesses the language of a previously unseen text. For simplicity, work with a single character encoding and just a few languages.__

In [173]:
def guessLanguage(text):
    # tokens = nltk.wordpunct_tokenize(text)
    # text = nltk.Text(tokens)
    text_trichar = ngrams(text, 6)
    freq_char_source = nltk.FreqDist(text_trichar)
    best_guess = ('', 0)
    best_intersection = []
    for lang in nltk.corpus.udhr.fileids():
        if (lang.endswith('-Latin1')):
            lang_trichar = ngrams(udhr.raw(lang).replace("\n", ""), 6)
            freq_char_cand = nltk.FreqDist(lang_trichar)
            intersection = list(set(freq_char_source.keys()) & set(freq_char_cand.keys()))
            tuples_source = []
            tuples_cand = []
            for char in intersection:
                tuples_source.append((char, freq_char_source[char]))
                tuples_cand.append((char, freq_char_cand[char]))
            spearman = nltk.spearman_correlation(tuples_source, tuples_cand)
            if ((best_guess[1] == 0 and spearman != 0.0) or (spearman != 0.0 and spearman > best_guess[1])):
                best_guess = (lang[:-7], spearman)
    return best_guess[0];

In [181]:
guessLanguage("You will definitely gues that the text is written in English")

'Achehnese'